# Selection rules

When considering the interaction of light and matter and determining the excitations an electron can undergo by absorption of one photon, the following selection rules are introduced for atoms

\begin{align*}
%&\Delta n  =\textrm{integer} \\ 
&\Delta l  = \pm1\\ 
&\Delta L  =0,\pm1 \\ 
&\Delta S  =0 \\ 
&\Delta J  =0,\pm 1 \quad (J=0 \Rightarrow \Delta J=0)
\end{align*}

for transitions involving changing the electronic state of an atom. Pure electronic excitations can be found in wave lengths of visible light and below. The selection rules can be understood by group theoretical considerations. If the atom (or, more generally, a molecule, for which selection rules can be found in similar manners) interacts with an electric field of incoming photons, electric dipole transitions to higher states are made possible. The probability of a transition from an initial state $\psi_i$ to a final state $\psi_f$ can be calculated as

$$
p_{fi}=\hat{\mu}_{fi}^2= \left| \langle \psi_f | \hat{\mu} | \psi_i \rangle \right|^2
$$

where $\hat{\mu}$ is the electric dipole moment. Again, this integral is by necessity zero if the integrand does not contain the totally symmetric irrep of the point group. In other words, we need to determine the symmetry of the function $\psi_f^{\ast} \hat{\mu} \psi_i$, by taking the direct product of the symmetry species of each function and see if this contains the totally symmetric irrep.

First we must then learn how to determine the symmetry of an electronic state. This is done by identifying all unpaired electrons in the molecule and take the direct product of the corresponding irreps of the molecular orbitals. Note here that the ground state of a closed-shell molecule always belong to the totally symmetric irrep (as all electrons are paired). Remembering that for two functions the symmetry species of the functions much match to acquire the totally symmetric irrep, so this transition is only achievable if

$$
D_{\mu i}(g)=D_{\mu}(g) \otimes D_i(g) \qquad \textrm{has the same symmetry as} \qquad D_f(g)
$$

Consider water with the basis set $(s_H,s_H',s_O,p_x,p_y,p_z)$. Using this it is possible to construct molecular orbitals by use of SALCs such that we obtain three orbitals of $A_1$ symmetry (the totally symmetrical symmetry), two of $B_1$ symmetry and one of $B_2$ symmetry. The ground state is then found to be $(1A_1)^2(1B_1)^2(2A_1)^4(1B_2)^2$ and is, as stated above, of $A_2$ symmetry. The closest antibonding orbital is $(3A_1)$. Further, it can be shown that x-polarized light is of $B_1$ symmetry, y-polarized of $B_2$ symmetry and z-polarized of $A_1$ symmetry.

We are thus prepared to conclude which transitions that can be achieved with which polarizations of incoming light. The first excited state yields the unpaired electrons in $(B_2)(A_1)$, for which the direct product of the two symmetries yields $B_2 \otimes A_1=B_2$. This symmetry matches that of y-polarized light, so this excitation is possible by use of y-polarized light. The second excited state is $(B_2)(B_1)$, with $B_2 \otimes B_1=A_2$. This symmetry does not match that of any polarization of light, and is thus inaccessible. This procedure can be carried out for all possible excitations.